# 是否需要标准化
需要。涉及到距离计算的时候会将不同的特征相同对待, 这时候就需要把他们变换到大致相同的范围

# KNN在分类时，是如何预测概率的？
uniform:
直接查个数, 然后除以总数

distance:
计算权重, 具体计算方法课上已经讲了, 然后计算每个类别的加权, 除以总数的加权

# 使用KNN算法建模，并预测一个人的收入是否会超过5万美元

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv('adult.csv', encoding='utf-8')
print(data.shape)
data.head()

(48842, 15)


age  workclass  fnlwgt     education  educational-num      marital-status  \
0   25    Private  226802          11th                7       Never-married   
1   38    Private   89814       HS-grad                9  Married-civ-spouse   
2   28  Local-gov  336951    Assoc-acdm               12  Married-civ-spouse   
3   44    Private  160323  Some-college               10  Married-civ-spouse   
4   18          ?  103497  Some-college               10       Never-married   

          occupation relationship   race  gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black    Male             0             0   
1    Farming-fishing      Husband  White    Male             0             0   
2    Protective-serv      Husband  White    Male             0             0   
3  Machine-op-inspct      Husband  Black    Male          7688             0   
4                  ?    Own-child  White  Female             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
1              50  United-States  <=50K  
2              40  United-States   >50K  
3              40  United-States   >50K  
4              30  United-States  <=50K

In [2]:
for column in data.columns:
    data[column] = data[column].replace({"?": np.nan})

In [3]:
data.dropna(inplace=True)

In [4]:
data.shape

(45222, 15)

In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['income'] = le.fit_transform(data['income'])

In [6]:
data.drop('educational-num', axis=1, inplace=True)
data.head()

age  workclass  fnlwgt     education      marital-status  \
0   25    Private  226802          11th       Never-married   
1   38    Private   89814       HS-grad  Married-civ-spouse   
2   28  Local-gov  336951    Assoc-acdm  Married-civ-spouse   
3   44    Private  160323  Some-college  Married-civ-spouse   
5   34    Private  198693          10th       Never-married   

          occupation   relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct      Own-child  Black   Male             0             0   
1    Farming-fishing        Husband  White   Male             0             0   
2    Protective-serv        Husband  White   Male             0             0   
3  Machine-op-inspct        Husband  Black   Male          7688             0   
5      Other-service  Not-in-family  White   Male             0             0   

   hours-per-week native-country  income  
0              40  United-States       0  
1              50  United-States       0  
2              40  United-States       1  
3              40  United-States       1  
5              30  United-States       0

In [7]:
columns = ["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country"]
dummy = pd.get_dummies(data[columns])
dummy

workclass_Federal-gov  workclass_Local-gov  workclass_Private  \
0                          0                    0                  1   
1                          0                    0                  1   
2                          0                    1                  0   
3                          0                    0                  1   
5                          0                    0                  1   
...                      ...                  ...                ...   
48837                      0                    0                  1   
48838                      0                    0                  1   
48839                      0                    0                  1   
48840                      0                    0                  1   
48841                      0                    0                  0   

       workclass_Self-emp-inc  workclass_Self-emp-not-inc  \
0                           0                           0   
1                           0                           0   
2                           0                           0   
3                           0                           0   
5                           0                           0   
...                       ...                         ...   
48837                       0                           0   
48838                       0                           0   
48839                       0                           0   
48840                       0                           0   
48841                       1                           0   

       workclass_State-gov  workclass_Without-pay  education_10th  \
0                        0                      0               0   
1                        0                      0               0   
2                        0                      0               0   
3                        0                      0               0   
5                        0                      0               1   
...                    ...                    ...             ...   
48837                    0                      0               0   
48838                    0                      0               0   
48839                    0                      0               0   
48840                    0                      0               0   
48841                    0                      0               0   

       education_11th  education_12th  ...  native-country_Portugal  \
0                   1               0  ...                        0   
1                   0               0  ...                        0   
2                   0               0  ...                        0   
3                   0               0  ...                        0   
5                   0               0  ...                        0   
...               ...             ...  ...                      ...   
48837               0               0  ...                        0   
48838               0               0  ...                        0   
48839               0               0  ...                        0   
48840               0               0  ...                        0   
48841               0               0  ...                        0   

       native-country_Puerto-Rico  native-country_Scotland  \
0                               0                        0   
1                               0                        0   
2                               0                        0   
3                               0                        0   
5                               0                        0   
...                           ...                      ...   
48837                           0                        0   
48838                           0                        0   
48839                           0                        0   
48840                           0                        0   
48841                           0            

In [8]:
data2 = pd.concat([data[["age", "fnlwgt", "capital-gain", "capital-loss", "hours-per-week", "income"]], dummy], axis=1)
data2

age  fnlwgt  capital-gain  capital-loss  hours-per-week  income  \
0       25  226802             0             0              40       0   
1       38   89814             0             0              50       0   
2       28  336951             0             0              40       1   
3       44  160323          7688             0              40       1   
5       34  198693             0             0              30       0   
...    ...     ...           ...           ...             ...     ...   
48837   27  257302             0             0              38       0   
48838   40  154374             0             0              40       1   
48839   58  151910             0             0              40       0   
48840   22  201490             0             0              20       0   
48841   52  287927         15024             0              40       1   

       workclass_Federal-gov  workclass_Local-gov  workclass_Private  \
0                          0                    0                  1   
1                          0                    0                  1   
2                          0                    1                  0   
3                          0                    0                  1   
5                          0                    0                  1   
...                      ...                  ...                ...   
48837                      0                    0                  1   
48838                      0                    0                  1   
48839                      0                    0                  1   
48840                      0                    0                  1   
48841                      0                    0                  0   

       workclass_Self-emp-inc  ...  native-country_Portugal  \
0                           0  ...                        0   
1                           0  ...                        0   
2                           0  ...                        0   
3                           0  ...                        0   
5                           0  ...                        0   
...                       ...  ...                      ...   
48837                       0  ...                        0   
48838                       0  ...                        0   
48839                       0  ...                        0   
48840                       0  ...                        0   
48841                       1  ...                        0   

       native-country_Puerto-Rico  native-country_Scotland  \
0                               0                        0   
1                               0                        0   
2                               0                        0   
3                               0                        0   
5                               0                        0   
...                           ...                      ...   
48837                           0                        0   
48838                           0                        0   
48839                           0                        0   
48840                           0                        0   
48841                           0                        0   

       native-country_South  native-country_Taiwan  native-country_Thailand  \
0                         0                      0                        0   
1                         0                      0                        0   
2                         0                      0                        0   
3                         0                      0                        0   
5                         0                      0                        0   
...                     ...                    ...                      ...   
48837                     0                      0                        0   
48838                     0                      0                        0   
48839                     0             

In [9]:
# 连续变量标准化
from sklearn.preprocessing import MinMaxScaler

y3 = data2['income']
X3 = data2.drop('income', axis=1)

scaler = MinMaxScaler()
X_scale = scaler.fit_transform(X3)
# display(X_scale)

In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X_scale, y3, test_size=0.25, random_state=0)

knn = KNeighborsClassifier(n_neighbors=5, weights="uniform")
knn.fit(X_train, y_train)
y_hat = knn.predict(X_test)
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88      8467
           1       0.66      0.57      0.61      2839

    accuracy                           0.82     11306
   macro avg       0.76      0.74      0.75     11306
weighted avg       0.81      0.82      0.81     11306



In [14]:
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()

grid = {"n_neighbors": range(2, 7), "weights": ['uniform', 'distance']}
gs = GridSearchCV(estimator=knn, param_grid=grid, scoring="accuracy")
gs.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': range(2, 7),
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [15]:
estimator = gs.best_estimator_
estimator

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [16]:
y_hat = estimator.predict(X_test)
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           0       0.85      0.93      0.89      8467
           1       0.70      0.50      0.58      2839

    accuracy                           0.82     11306
   macro avg       0.77      0.71      0.73     11306
weighted avg       0.81      0.82      0.81     11306

